In [1]:
import pickle
import warnings
import datetime
import numpy as np
import pandas as pd
import csv
import time

import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [3]:

" Offset value should be based on the size of player detection "
offset = 10 # temp value

line_thickness = 2
line_color = (0, 255, 0)  # Green color
line_color_red = (0, 0, 255)  # Red color
line_color_blue = (255, 0, 0)  # Blue color


# Lineguide

In [8]:
def draw_horizontal_panel(image, shoulderR, shoulderL):
    height, width, _ = image.shape
    
    left_line = (int(shoulderL.x * width) - offset, 0), (int(shoulderL.x * width) - offset, height)
    right_line = (int(shoulderR.x * width) + offset, 0), (int(shoulderR.x * width) + offset, height)
    
    cv2.line(image, left_line[0], left_line[1], line_color, line_thickness)
    cv2.line(image, right_line[0], right_line[1], line_color, line_thickness)

    return left_line, right_line


In [ ]:
def draw_vertical_panel(image, nose):
    height, width, _ = image.shape

    noseY = int(nose.y * height)
    noseX =  int(nose.x * width)

    top_line = (0, noseY), (width, noseY)
    bottom_line = (0, (noseY - 0))

    # Check outside frame must have minus
    print((noseY + 100) - int(width)) 

    cv2.line(image, top_line[0], top_line[1], line_color_blue, line_thickness)

# Gap Of Each Joints

In [10]:
def draw_line_and_calculate_gap(image, start_point, end_point):
    if start_point and end_point:
        x1, y1 = start_point.x * image.shape[1], start_point.y * image.shape[0]
        x2, y2 = end_point.x * image.shape[1], end_point.y * image.shape[0]
        
        # Calculate normalized coordinates
        gap_x = (x2 - x1) / image.shape[1]
        gap_y = (y2 - y1) / image.shape[0]

        # Draw line
        x1, y1 = int(x1), int(y1)
        x2, y2 = int(x2), int(y2)
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

        gap_landmark = landmark_pb2.NormalizedLandmark()
        gap_landmark.x = gap_x
        gap_landmark.y = gap_y

        return gap_landmark
    else:
        return None

# Prepare The Features

In [11]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Get specific landmarks
        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
        wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_WRIST]
        elbow_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_ELBOW]
        wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
        elbow_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_ELBOW]

        # Use For Making Guideline Purpose
        shoulder_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_SHOULDER]
        shoulder_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_SHOULDER]

        show_landmark_list = landmark_pb2.NormalizedLandmarkList()
        show_landmark_list.landmark.extend([nose, wrist_l, wrist_r, elbow_l, elbow_r])
        
        # Draw landmarks
        for landmark in show_landmark_list.landmark:
            x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
            cv2.circle(image, (x, y), 5, (255, 0, 0), -1)

        """
        IT GIVES THE LOCATION OF THE CENTER BASED OFF 
        THE NOSE X,Y FROM THE FRAMES
        """

        # nose_x, nose_y = int(nose.x * image.shape[1]), int(nose.y * image.shape[0])
        
        wristL_x, wristL_y = int(wrist_l.x * image.shape[1]), int(wrist_l.y * image.shape[0])
        wristR_x, wristR_y = int(wrist_r.x * image.shape[1]), int(wrist_r.y * image.shape[0])
        
        left_line, right_line = draw_horizontal_panel(image, shoulder_l, shoulder_r)
        top_line = draw_vertical_panel(image, nose)
        # Below this is temporary for drawing the line, and we need the calculation of the gap

        wristL_gap = (wristL_x, wristL_y), (left_line[0][0], wristL_y)
        wristR_gap = (wristR_x, wristR_y), (right_line[0][0], wristR_y)

        left_line_x, left_line_y = wristL_gap[1]
        right_line_x, right_line_y = wristR_gap[1]

        """ Wrist Left """
        
        wristL_x, wristL_y = wristL_gap[0]

        # Horizontal Gap
        # make it so it has a (+ and - value)
        wristL_leftLine = wristL_x - left_line_x
        wristL_rightLine = wristL_x -right_line_x
         

        cv2.line(image, wristL_gap[0], wristL_gap[1], line_color_blue, line_thickness)
        cv2.line(image, wristL_gap[0], (right_line[0][0], wristL_y + 20), line_color_red, line_thickness)
        
        wristL_landmark = landmark_pb2.NormalizedLandmark()
        wristL_landmark.x = wristL_leftLine
        wristL_landmark.y = wristL_rightLine

        # Vertical Gap
         

        """ Wrist Right """

        wristR_x, wristR_y = wristR_gap[0]

        # Horizontal Gap
        wristR_leftLine = wristR_x - left_line_x
        wristR_rightLine = wristR_x - right_line_x

        cv2.line(image, wristR_gap[0], wristR_gap[1] , line_color_blue, line_thickness)
        cv2.line(image, wristR_gap[0], (left_line[0][0], wristR_y + 20) , line_color_red, line_thickness)
        
        wristR_landmark = landmark_pb2.NormalizedLandmark()
        wristR_landmark.x = wristR_leftLine
        wristR_landmark.y = wristR_rightLine # it actually x too, instead of y for the purpose of testing

        """ ============== MAKE NEW LANDMARK PB2 ==================== """
        
        " =========================================================== "

        # Drawing line and calculating gap for left wrist
        gap_nose_left = draw_line_and_calculate_gap(image, nose, wrist_l)

        # Drawing line and calculating gap for right wrist
        gap_nose_right = draw_line_and_calculate_gap(image, nose, wrist_r)

        gap_hand_left = draw_line_and_calculate_gap(image, elbow_l, wrist_l)
        gap_hand_right = draw_line_and_calculate_gap(image, elbow_r, wrist_r)
        
        # new_lm = landmark_pb2.NormalizedLandmarkList()
        # new_lm.landmark.extend([wrist_l, wrist_r, elbow_l, elbow_r, gap_nose_right, gap_nose_left])

        new_lm = landmark_pb2.NormalizedLandmarkList()
        new_lm.landmark.extend([wrist_l, wrist_r, elbow_l, elbow_r, gap_nose_right, gap_nose_left, wristL_landmark, wristR_landmark])

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [7]:
num_coords = len(new_lm.landmark)
num_coords

landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8']

# Create Empty Dataset

In [ ]:
file_csv = 'dataset/csv/v1_coords.csv'

with open(file_csv, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# Process Image From Image Dataset

In [ ]:
def process_images(image_path, holistic, class_name):
    # Read the image from the file path
    frame = cv2.imread(image_path)

    # Recolor Feed
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make Detections
    results = holistic.process(image)

    # Recolor image back to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Get specific landmarks
    nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
    wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_WRIST]
    elbow_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_ELBOW]
    wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
    elbow_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_ELBOW]

    # Create a custom landmark list to show
    show_landmark_list = landmark_pb2.NormalizedLandmarkList()
    show_landmark_list.landmark.extend([nose, wrist_l, wrist_r, elbow_l, elbow_r])

    # Draw landmarks
    for landmark in show_landmark_list.landmark:
        x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
        cv2.circle(image, (x, y), 5, (255, 0, 0), -1)

    """
    IT GIVES THE LOCATION OF THE CENTER BASED OFF 
    THE NOSE X,Y FROM THE FRAMES
    """

    nose_x, nose_y = int(nose.x * image.shape[1]), int(nose.y * image.shape[0])

    wristL_x, wristL_y = int(wrist_l.x * image.shape[1]), int(wrist_l.y * image.shape[0])
    wristR_x, wristR_y = int(wrist_r.x * image.shape[1]), int(wrist_r.y * image.shape[0])
        
    left_line, right_line = draw_horizontal_panel(image, nose_x, nose_y)
    # Below this is temporary for drawing the line, and we need the calculation of the gap

    wristL_gap = (wristL_x, wristL_y), (left_line[0][0], wristL_y)
    wristR_gap = (wristR_x, wristR_y), (right_line[0][0], wristR_y)

    left_line_x, left_line_y = wristL_gap[1]
    right_line_x, right_line_y = wristR_gap[1]

    """ Wrist Left """
        
    wristL_x, wristL_y = wristL_gap[0]

    # Horizontal Gap
    # make it so it has a (+ and - value)
    wristL_leftLine = wristL_x - left_line_x
    wristL_rightLine = wristL_x - right_line_x
         

    cv2.line(image, wristL_gap[0], wristL_gap[1], line_color_blue, line_thickness)
    cv2.line(image, wristL_gap[0], (right_line[0][0], wristL_y + 20), line_color_red, line_thickness)
        
    wristL_landmark = landmark_pb2.NormalizedLandmark()
    wristL_landmark.x = wristL_leftLine
    wristL_landmark.y = wristL_rightLine

    # Vertical Gap
         
    """ Wrist Right """

    wristR_x, wristR_y = wristR_gap[0]

    # Horizontal Gap
    wristR_leftLine = wristR_x - left_line_x
    wristR_rightLine = wristR_x - right_line_x

    cv2.line(image, wristR_gap[0], wristR_gap[1] , line_color_blue, line_thickness)
    cv2.line(image, wristR_gap[0], (left_line[0][0], wristR_y + 20) , line_color_red, line_thickness)
        
    wristR_landmark = landmark_pb2.NormalizedLandmark()
    wristR_landmark.x = wristR_leftLine
    wristR_landmark.y = wristR_rightLine # it actually x too, instead of y for the purpose of testing

    # Drawing line and calculating gap for left wrist
    gap_nose_left = draw_line_and_calculate_gap(image, nose, wrist_l)

    # Drawing line and calculating gap for right wrist
    gap_nose_right = draw_line_and_calculate_gap(image, nose, wrist_r)


    new_lm = landmark_pb2.NormalizedLandmarkList()
    new_lm.landmark.extend([wrist_l, wrist_r, elbow_l, elbow_r, gap_nose_right, gap_nose_left, wristL_landmark, wristR_landmark])
    
    # Extract Pose landmarks
    pose = new_lm.landmark
    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

    # Append class name 
    pose_row.insert(0, class_name)

    # Export to CSV
    with open(file_csv, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(pose_row)
    

In [15]:
def process_images_in_folder(folder_path, holistic, class_name):
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Process each image in the folder
            image_path = os.path.join(folder_path, filename)
            process_images(image_path, holistic, class_name)

# Extraction Image To Coordinate Dataset

In [ ]:
"""
Written Rule :
" Pose Detection Value " : "Location Of Images Dataset"
"""

folders = {
    "jab": ["guard/jab", "unguard/jab"],
    "straight": ["guard/straight", "unguard/straight"],
    "hook_l": ["guard/hook_left", "unguard/hook_left"],
    "hook_r": ["guard/hook_right", "unguard/hook_right"],
    "uppercut_l": ["guard/uppercut_left", "unguard/uppercut_left"],
    "uppercut_r": ["guard/uppercut_right", "unguard/uppercut_right"],
    "guard": ["guard/guard"],
    "idle": ["unguard/idle"]
    } 

# Initialize mediapipe Holistic
mp_holistic = mp.solutions.holistic.Holistic()

for class_name, subfolders in folders.items():
    for subfolder in subfolders:
        image_folder = f"dataset/images/{subfolder}"
        print(f"Class Name: {class_name}, Image Folder: {image_folder}")
        try:
            process_images_in_folder(image_folder, mp_holistic, class_name)
        except:
            pass

mp_holistic.close()